### Imports

In [1]:
from joblib import load
import numpy as np
import pandas as pd

In [2]:
model = load('model.pkl')

model

Pipeline(steps=[('enc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['smoker', 'sex', 'region']),
                                                 ('num', RobustScaler(),
                                                  ['age', 'bmi'])])),
                ('poly', PolynomialFeatures()),
                ('model',
                 Lasso(alpha=0.9, positive=True, precompute=True,
                       warm_start=True))])

### Fonction modèle

In [3]:
def prime_assurance(model, age, sex, bmi, children, smoker, region) :
    x = np.array([age, sex, bmi, children, smoker, region])

    df = pd.DataFrame(x.reshape(-1, 6), columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])

    print(model.predict(df))

In [4]:
def score (a, b):
    score = a / b
    score = score * 100
    score = round(score, 2)

    return f"Score : {score}%"

### Tests du modèle

In [5]:
y = np.array([19, "female", 27.9, 0, "yes", "southwest"])
pd.DataFrame(y.reshape(-1, 6), columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])

,age,sex,bmi,children,smoker,region
0,19,female,27.9,0,yes,southwest


In [6]:
prime_assurance(model, 19, "female", 27.9, 0, "Oui", "southwest")
prime_assurance(model, 18, "male", 33.77, 1, "Non", "southeast")
prime_assurance(model, 28, "male", 33, 3, "Non", "southeast")

[23681.53360447]
[3019.02021517]
[5846.85770764]


In [7]:
print("Dataset 90/10,", score(16884.92400, 23795.00657263))
print("Dataset 95/5,", score(16884.92400, 23681.53360447))

Dataset 90/10, Score : 70.96%
Dataset 95/5, Score : 71.3%


In [8]:
print("Dataset 90/10,", score(1725.55230, 3247.73081881))
print("Dataset 95/5,", score(1725.55230, 3019.02021517))

Dataset 90/10, Score : 53.13%
Dataset 95/5, Score : 57.16%


In [9]:
print("Dataset 90/10,", score(4449.46200, 5936.3144794))
print("Dataset 95/5,", score(4449.46200, 5846.85770764))

Dataset 90/10, Score : 74.95%
Dataset 95/5, Score : 76.1%
